# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('LogReg').getOrCreate()

In [6]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [7]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [8]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [10]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [14]:
df.select(['Onboard_date', 'Location', 'Company', 'Churn']).show()

+-------------------+--------------------+--------------------+-----+
|       Onboard_date|            Location|             Company|Churn|
+-------------------+--------------------+--------------------+-----+
|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|2016-01-19 15:31:15|765 Tricia Row Ka...|          Love-Jones|    1|
|2009-03-03 23:13:37|6187 Olson Mounta...|        Kelly-Warren|    1|
|2016-12-05 03:35:43|4846 Savannah Roa...|   Reynolds-Sheppard|    1|
|2006-03-09 14:50:20|25271 Roy Express...|          Singh-Cole|    1|
|2011-09-29 05:47:23|3725 Caroline Str...|           Lopez PLC|    1|
|2006-03-28 15:42:45|363 Sandra Lodge ...|       Reed-Martinez|    1|
|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...|    1|
|2015-05-28 12:14:03

In [13]:
from pyspark.ml.pipeline import Pipeline

from pyspark.ml.feature import VectorAssembler

In [15]:
Vect_Feature = VectorAssembler(inputCols=['Age', 'Total_Purchase', 
                                          'Account_Manager', 'Years', 
                                          'Num_Sites'], outputCol='features')

In [22]:
dfn = Vect_Feature.transform(df)

In [23]:
df_n = dfn.select(['features', 'Churn'])

In [25]:
df_n.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [26]:
from pyspark.ml.classification import LogisticRegression

In [27]:
train_df, tst_df = df_n.randomSplit([0.7, 0.3])

In [28]:
train_df.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                631|
|   mean|0.16323296354992076|
| stddev| 0.3698712876817587|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [29]:
tst_df.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                269|
|   mean|0.17472118959107807|
| stddev| 0.3804362386474134|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [31]:
model = LogisticRegression(featuresCol='features', labelCol='Churn', predictionCol='prediction')

In [32]:
trained = model.fit(train_df)

In [33]:
result = trained.evaluate(tst_df)

In [34]:
result.accuracy

0.8884758364312267

In [50]:
result.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[1.21176013528534...|[0.77061023652597...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.77085844181409...|[0.99885460359275...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.97132312607063...|[0.99745562628109...|       0.0|
|[28.0,11245.38,0....|    0|[4.15211518045634...|[0.98451252794298...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.43184780729417...|[0.98824727489681...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.07702314662677...|[0.99380022464341...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.36348180862724...|[0.99827960898761...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.42653746298658...|[0.96852368144363...|       0.0|
|[30.0,10744.14,1....|    1|[2.24688700599606...|[0.90438167617939...|       0.0|
|[30.0,11575.37,

In [51]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [52]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [55]:
accu = evaluator.evaluate(result.predictions)

In [56]:
accu

0.764711520030669

In [35]:
df_test = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [36]:
df_test.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [38]:
df_test.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [39]:
AssembledVect = VectorAssembler(inputCols=['Age', 'Total_Purchase', 
                                          'Account_Manager', 'Years', 
                                          'Num_Sites'], outputCol='features')

In [40]:
df_test_new = AssembledVect.transform(df_test)

In [41]:
df_test_ready = df_test_new.select('features')

In [42]:
df_test_ready.show()

+--------------------+
|            features|
+--------------------+
|[37.0,9935.53,1.0...|
|[23.0,7526.94,1.0...|
|[65.0,100.0,1.0,1...|
|[32.0,6487.5,0.0,...|
|[32.0,13147.71,1....|
|[22.0,8445.26,1.0...|
+--------------------+



In [58]:
final_train = model.fit(df_n)

In [60]:
Test_Result = final_train.transform(df_test_ready)

In [61]:
Test_Result.summary().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|                 6|
|   mean|0.6666666666666666|
| stddev|0.5163977794943223|
|    min|               0.0|
|    25%|               0.0|
|    50%|               1.0|
|    75%|               1.0|
|    max|               1.0|
+-------+------------------+



In [62]:
Test_Result.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|       1.0|
|[65.0,100.0,1.0,1...|[-3.7691621189402...|[0.02255110110413...|       1.0|
|[32.0,6487.5,0.0,...|[-5.0956222016519...|[0.00608622642084...|       1.0|
|[32.0,13147.71,1....|[1.10475867224153...|[0.75115067517475...|       0.0|
|[22.0,8445.26,1.0...|[-1.6896019277064...|[0.15582819767635...|       1.0|
+--------------------+--------------------+--------------------+----------+



In [80]:
arr = [1,2,3,4,5,6,7,8]

In [89]:
def shift(arr, d):
#     a = []
#     b = []
#     for i in range(d):
#         a.append(arr[i])
        
#     for j in range(d, len(arr)):
#         b.append(arr[j]) 
#     b.extend(a)
    a = arr[d:]
    a.extend(arr[:d])
    return a

In [90]:
shift(arr, 2)

In [95]:
arr is None

False